In [ ]:
# Copyright (c) 2024 Microsoft Corporation.
# Licensed under the MIT License.

# Neo4j Import of GraphRAG Result Parquet files

This notebook imports the results of the GraphRAG indexing process into the Neo4j Graph database for further processing, analysis or visualization. 

You can also build your own GenAI applications using Neo4j and a number of RAG strategies with LangChain, LlamaIndex, Haystack, and many other frameworks.
See: https://neo4j.com/labs/genai-ecosystem

Here is what the end result looks like:

![](https://dev.assets.neo4j.com/wp-content/uploads/graphrag-neo4j-visualization.png)

## How does it work?

The notebook loads the parquet files from the `output` folder of your indexing process and loads them into Pandas dataframes.
It then uses a batching approach to send a slice of the data into Neo4j to create nodes and relationships and add relevant properties. The id-arrays on most entities are turned into relationships. 

All operations use MERGE, so they are idempotent, and you can run the script multiple times.

If you need to clean out the database, you can run the following statement

```cypher
MATCH (n)
CALL { WITH n DETACH DELETE n } IN TRANSACTIONS OF 25000 ROWS;
```

In [1]:
from pathlib import Path

print(Path.cwd())
GRAPHRAG_FOLDER = "../../inputs/artifacts"

d:\programming_works\vscodeworks\graph-rag-dev\examples_notebooks\community_contrib\neo4j


### Depedendencies

We only need Pandas and the neo4j Python driver with the rust extension for faster network transport.

In [ ]:
%poetry add pandas neo4j-rust-ext

In [2]:
import time

import pandas as pd
from neo4j import GraphDatabase

## Neo4j Installation

You can create a free instance of Neo4j [online](https://console.neo4j.io). You get a credentials file that you can use for the connection credentials. You can also get an instance in any of the cloud marketplaces.

If you want to install Neo4j locally either use [Neo4j Desktop](https://neo4j.com/download) or 
the official Docker image: `docker run -e NEO4J_AUTH=neo4j/password -p 7687:7687 -p 7474:7474 neo4j` 

In [3]:
NEO4J_URI = "neo4j://localhost"  # or neo4j+s://xxxx.databases.neo4j.io
NEO4J_USERNAME = "neo4j"
NEO4J_PASSWORD = "password"  # your password  # noqa: S105
NEO4J_DATABASE = "neo4j"

# Create a Neo4j driverdo
driver = GraphDatabase.driver(NEO4J_URI, auth=(NEO4J_USERNAME, NEO4J_PASSWORD))



In [5]:
def get_data_info(tx):
    result = tx.run("MATCH (n) RETURN count(n) AS node_count")
    node_count = result.single()["node_count"]
    
    result = tx.run("MATCH ()-[r]->() RETURN count(r) AS relationship_count")
    relationship_count = result.single()["relationship_count"]
    
    return node_count, relationship_count

def clear_data(tx):
    tx.run("MATCH (n) DETACH DELETE n")

with driver.session(database=NEO4J_DATABASE) as session:
    node_count, relationship_count = session.execute_read(get_data_info)
    
    if node_count > 0 or relationship_count > 0:
        print(f"Database contains {node_count} nodes and {relationship_count} relationships.")
        session.execute_write(clear_data)
        print("All data has been cleared.")
    else:
        print("Database is already empty.")

Database is already empty.


## Batched Import

The batched import function takes a Cypher insert statement (needs to use the variable `value` for the row) and a dataframe to import.
It will send by default 1k rows at a time as query parameter to the database to be inserted.

In [6]:
def batched_import(statement, df, batch_size=1000):
    """
    Import a dataframe into Neo4j using a batched approach.

    Parameters: statement is the Cypher query to execute, df is the dataframe to import, and batch_size is the number of rows to import in each batch.
    """
    total = len(df)
    start_s = time.time()
    for start in range(0, total, batch_size):
        batch = df.iloc[start : min(start + batch_size, total)]
        result = driver.execute_query(
            "UNWIND $rows AS value " + statement,
            rows=batch.to_dict("records"),
            database_=NEO4J_DATABASE,
        )
        print(result.summary.counters)
    print(f"{total} rows in {time.time() - start_s} s.")
    return total

## Indexes and Constraints

Indexes in Neo4j are only used to find the starting points for graph queries, e.g. quickly finding two nodes to connect.
Constraints exist to avoid duplicates, we create them mostly on id's of Entity types.

We use some Types as markers with two underscores before and after to distinguish them from the actual entity types.

The default relationship type here is `RELATED` but we could also infer a real relationship-type from the description or the types of the start and end-nodes.

* `__Entity__`
* `__Document__`
* `__Chunk__`
* `__Community__`
* `__Covariate__`

In [7]:
# create constraints, idempotent operation

statements = """
create constraint chunk_id if not exists for (c:__Chunk__) require c.id is unique;
create constraint document_id if not exists for (d:__Document__) require d.id is unique;
create constraint entity_id if not exists for (c:__Community__) require c.community is unique;
create constraint entity_id if not exists for (e:__Entity__) require e.id is unique;
create constraint entity_title if not exists for (e:__Entity__) require e.name is unique;
create constraint entity_title if not exists for (e:__Covariate__) require e.title is unique;
create constraint related_id if not exists for ()-[rel:RELATED]->() require rel.id is unique;
""".split(";")

for statement in statements:
    if len((statement or "").strip()) > 0:
        print(statement)
        driver.execute_query(statement)


create constraint chunk_id if not exists for (c:__Chunk__) require c.id is unique

create constraint document_id if not exists for (d:__Document__) require d.id is unique

create constraint entity_id if not exists for (c:__Community__) require c.community is unique

create constraint entity_id if not exists for (e:__Entity__) require e.id is unique

create constraint entity_title if not exists for (e:__Entity__) require e.name is unique

create constraint entity_title if not exists for (e:__Covariate__) require e.title is unique

create constraint related_id if not exists for ()-[rel:RELATED]->() require rel.id is unique


## Import Process

### Importing the Documents

We're loading the parquet file for the documents and create nodes with their ids and add the title property.
We don't need to store text_unit_ids as we can create the relationships and the text content is also contained in the chunks.

In [8]:
doc_df = pd.read_parquet(
    f"{GRAPHRAG_FOLDER}/create_final_documents.parquet", columns=["id", "title"]
)
doc_df.head(2)

,id,title
0,d2118a27ade083d9ba57d266aaf1dc5f,combined.txt


In [9]:
# Import documents
statement = """
MERGE (d:__Document__ {id:value.id})
SET d += value {.title}
"""

batched_import(statement, doc_df)

{'_contains_updates': True, 'labels_added': 1, 'nodes_created': 1, 'properties_set': 2}
1 rows in 0.2913062572479248 s.


1

### Loading Text Units

We load the text units, create a node per id and set the text and number of tokens.
Then we connect them to the documents that we created before.

In [10]:
text_df = pd.read_parquet(
    f"{GRAPHRAG_FOLDER}/create_final_text_units.parquet",
    columns=["id", "text", "n_tokens", "document_ids"],
)
text_df.head(2)

,id,text,n_tokens,document_ids
0,2708e6738dd5195e194421771a53b40a,华中科技大学信息安全202101班2024-2025年度第一学期课表|Unnamed: 1|...,1200,[d2118a27ade083d9ba57d266aaf1dc5f]
1,ca786eb823b82150e93843485960fd81,安基地B105\n 云计算与虚拟化 10-15周 网安基地C103| | | | | | |...,1200,[d2118a27ade083d9ba57d266aaf1dc5f]


In [11]:
statement = """
MERGE (c:__Chunk__ {id:value.id})
SET c += value {.text, .n_tokens}
WITH c, value
UNWIND value.document_ids AS document
MATCH (d:__Document__ {id:document})
MERGE (c)-[:PART_OF]->(d)
"""

batched_import(statement, text_df)

{'_contains_updates': True, 'labels_added': 19, 'relationships_created': 19, 'nodes_created': 19, 'properties_set': 57}
19 rows in 0.16123485565185547 s.


19

### Loading Nodes

For the nodes we store id, name, description, embedding (if available), human readable id.

In [12]:
entity_df = pd.read_parquet(
    f"{GRAPHRAG_FOLDER}/create_final_entities.parquet",
    columns=[
        "name",
        "type",
        "description",
        "human_readable_id",
        "id",
        "description_embedding",
        "text_unit_ids",
    ],
)
entity_df.head(2)

,name,type,description,human_readable_id,id,description_embedding,text_unit_ids
0,信息安全202101班,ACADEMIC_GROUP,A specific class focusing on information secur...,0,b45241d70f0e43fca764df95b2b81f77,"[-0.019470215, 0.009162903, 0.017608643, 0.010...",[2708e6738dd5195e194421771a53b40a]
1,2024-2025年度第一学期,TIME_PERIOD,"The entity referred to as ""2024-2025年度第一学期"" co...",1,4119fd06010c494caa07f439b333f4c5,"[0.0072250366, -0.014083862, 0.0044784546, 0.0...","[2708e6738dd5195e194421771a53b40a, 3616bb01774..."


In [13]:
entity_statement = """
MERGE (e:__Entity__ {id:value.id})
SET e += value {.human_readable_id, .description, name:replace(value.name,'"','')}
WITH e, value
CALL db.create.setNodeVectorProperty(e, "description_embedding", value.description_embedding)
CALL apoc.create.addLabels(e, case when coalesce(value.type,"") = "" then [] else [apoc.text.upperCamelCase(replace(value.type,'"',''))] end) yield node
UNWIND value.text_unit_ids AS text_unit
MATCH (c:__Chunk__ {id:text_unit})
MERGE (c)-[:HAS_ENTITY]->(e)
"""

batched_import(entity_statement, entity_df)

{'_contains_updates': True, 'labels_added': 112, 'relationships_created': 396, 'nodes_created': 112, 'properties_set': 448}
112 rows in 0.8460993766784668 s.


112

### Import Relationships

For the relationships we find the source and target node by name, using the base `__Entity__` type.
After creating the `RELATED` relationships, we set the description as attribute.

In [14]:
rel_df = pd.read_parquet(
    f"{GRAPHRAG_FOLDER}/create_final_relationships.parquet",
    columns=[
        "source",
        "target",
        "id",
        "rank",
        "weight",
        "human_readable_id",
        "description",
        "text_unit_ids",
    ],
)
rel_df.head(2)

,source,target,id,rank,weight,human_readable_id,description,text_unit_ids
0,信息安全202101班,华中科技大学,482027a59f32484c9c44fd700615c1b6,14,9.0,0,The Information Security class of 202101 is pa...,[2708e6738dd5195e194421771a53b40a]
1,信息安全202101班,课程表,de837ff3d626451282ff6ac77a82216d,11,8.0,1,The course schedule is created for the specifi...,[2708e6738dd5195e194421771a53b40a]


In [15]:
rel_statement = """
    MATCH (source:__Entity__ {name:replace(value.source,'"','')})
    MATCH (target:__Entity__ {name:replace(value.target,'"','')})
    // not necessary to merge on id as there is only one relationship per pair
    MERGE (source)-[rel:RELATED {id: value.id}]->(target)
    SET rel += value {.rank, .weight, .human_readable_id, .description, .text_unit_ids}
    RETURN count(*) as createdRels
"""

batched_import(rel_statement, rel_df)

{'_contains_updates': True, 'relationships_created': 134, 'properties_set': 804}
134 rows in 0.204451322555542 s.


134

### Importing Communities

For communities we import their id, title, level.
We connect the `__Community__` nodes to the start and end nodes of the relationships they refer to.

Connecting them to the chunks they orignate from is optional, as the entites are already connected to the chunks.

In [16]:
community_df = pd.read_parquet(
    f"{GRAPHRAG_FOLDER}/create_final_communities.parquet",
    columns=["id", "level", "title", "text_unit_ids", "relationship_ids"],
)

community_df.head(2)

,id,level,title,text_unit_ids,relationship_ids
0,0,0,Community 0,"[2708e6738dd5195e194421771a53b40a, 2708e6738dd...","[482027a59f32484c9c44fd700615c1b6, de837ff3d62..."
1,8,0,Community 8,"[2708e6738dd5195e194421771a53b40a,3a7935cac78b...","[cccfa151fedc4b218a8d96adc7dceabe, ce54725672a..."


In [17]:
statement = """
MERGE (c:__Community__ {community:value.id})
SET c += value {.level, .title}
/*
UNWIND value.text_unit_ids as text_unit_id
MATCH (t:__Chunk__ {id:text_unit_id})
MERGE (c)-[:HAS_CHUNK]->(t)
WITH distinct c, value
*/
WITH *
UNWIND value.relationship_ids as rel_id
MATCH (start:__Entity__)-[:RELATED {id:rel_id}]->(end:__Entity__)
MERGE (start)-[:IN_COMMUNITY]->(c)
MERGE (end)-[:IN_COMMUNITY]->(c)
RETURn count(distinct c) as createdCommunities
"""

batched_import(statement, community_df)

{'_contains_updates': True, 'labels_added': 19, 'relationships_created': 274, 'nodes_created': 19, 'properties_set': 57}
19 rows in 0.19335722923278809 s.


19

### Importing Community Reports

Fo the community reports we create nodes for each communitiy set the id, community, level, title, summary, rank, and rank_explanation and connect them to the entities they are about.
For the findings we create the findings in context of the communities.

In [18]:
community_report_df = pd.read_parquet(
    f"{GRAPHRAG_FOLDER}/create_final_community_reports.parquet",
    columns=[
        "id",
        "community",
        "level",
        "title",
        "summary",
        "findings",
        "rank",
        "rank_explanation",
        "full_content",
    ],
)
community_report_df.head(2)

,id,community,level,title,summary,findings,rank,rank_explanation,full_content
0,d6e25ee8-5db6-423d-ac6d-392aa8785a7f,10,1,HuaZhong University of Science and Technology'...,The community is structured around the Network...,[{'explanation': 'The network security class o...,8.5,The rating reflects the community's strong ali...,# HuaZhong University of Science and Technolog...
1,d60750ba-8191-48ca-aad1-cdde6b649e77,11,1,HuaZhong University of Science and Technology'...,The community is centered around HuaZhong Univ...,[{'explanation': 'HuaZhong University of Scien...,8.0,The rating is high due to the comprehensive ra...,# HuaZhong University of Science and Technolog...


In [19]:
# Import communities
community_statement = """
MERGE (c:__Community__ {community:value.community})
SET c += value {.level, .title, .rank, .rank_explanation, .full_content, .summary}
WITH c, value
UNWIND range(0, size(value.findings)-1) AS finding_idx
WITH c, value, finding_idx, value.findings[finding_idx] as finding
MERGE (c)-[:HAS_FINDING]->(f:Finding {id:finding_idx})
SET f += finding
"""
batched_import(community_statement, community_report_df)

{'_contains_updates': True, 'labels_added': 103, 'relationships_created': 103, 'nodes_created': 103, 'properties_set': 423}
19 rows in 0.11231160163879395 s.


19

### Importing Covariates

Covariates are for instance claims on entities, we connect them to the chunks where they originate from.

you should enable the related config to have covariates ouput, check in "claim_extraction:"

In [21]:
cov_df = (pd.read_parquet(f"{GRAPHRAG_FOLDER}/create_final_covariates.parquet"))
# columns=["id","text_unit_id"])
cov_df.head(2)
# Subject id do not match entity ids

,id,human_readable_id,covariate_type,type,description,subject_id,subject_type,object_id,object_type,status,start_date,end_date,source_text,text_unit_id,document_ids,n_tokens
0,ccc984e7-bfc2-49bf-8da6-43ae2a35e659,1,claim,EDUCATIONAL,The university is offering a curriculum in inf...,HUAZHONG UNIVERSITY OF SCIENCE AND TECHNOLOGY,None,NONE,None,TRUE,2024-09-01T00:00:00,2025-01-31T00:00:00,华中科技大学信息安全202101班2024-2025年度第一学期课表,2708e6738dd5195e194421771a53b40a,[d2118a27ade083d9ba57d266aaf1dc5f],1200
1,618eaa9d-a96b-4265-ba77-1fcb5405fa4e,2,claim,EDUCATIONAL,"Mei Song, a lecturer at the university, is tea...",HUAZHONG UNIVERSITY OF SCIENCE AND TECHNOLOGY,None,MEI SONG,None,TRUE,2024-09-01T00:00:00,2025-01-31T00:00:00,"""网络安全程序设计 32/2 梅松（讲师（高校））""",2708e6738dd5195e194421771a53b40a,[d2118a27ade083d9ba57d266aaf1dc5f],1200


In [22]:
# Import covariates
cov_statement = """
MERGE (c:__Covariate__ {id:value.id})
SET c += apoc.map.clean(value, ["text_unit_id", "document_ids", "n_tokens"], [NULL, ""])
WITH c, value
MATCH (ch:__Chunk__ {id: value.text_unit_id})
MERGE (ch)-[:HAS_COVARIATE]->(c)
"""
batched_import(cov_statement, cov_df)

{'_contains_updates': True, 'labels_added': 91, 'relationships_created': 91, 'nodes_created': 91, 'properties_set': 1067}
91 rows in 0.12279939651489258 s.


91

### Visualize your data

You can now [Open] Neo4j on Aura, you need to log in with either SSO or your credentials.

Or open https://workspace-preview.neo4j.io and connect to your local instance, remember the URI is `neo4j://localhost` and `neo4j` as username and `password` as password.

In "Explore" you can explore by using visual graph patterns and then explore and expand further.

In "Query", you can open the left sidebar and explore by clicking on the nodes and relationships.
You can also use the co-pilot to generate Cypher queries for your, here are some examples.

#### Show a few `__Entity__` nodes and their relationships (Entity Graph)

```cypher
MATCH path = (:__Entity__)-[:RELATED]->(:__Entity__)
RETURN path LIMIT 200
```

#### Show the Chunks and the Document (Lexical Graph)

```cypher
MATCH (d:__Document__) WITH d LIMIT 1
MATCH path = (d)<-[:PART_OF]-(c:__Chunk__)
RETURN path LIMIT 100
```

####  Show a Community and it's Entities

```cypher
MATCH (c:__Community__) WITH c LIMIT 1
MATCH path = (c)<-[:IN_COMMUNITY]-()-[:RELATED]-(:__Entity__)
RETURN path LIMIT 100
```

#### Show everything

```cypher
MATCH (d:__Document__) WITH d LIMIT 1
MATCH path = (d)<-[:PART_OF]-(:__Chunk__)-[:HAS_ENTITY]->()-[:RELATED]-()-[:IN_COMMUNITY]->()
RETURN path LIMIT 250
```

We showed the visualization of this last query at the beginning.

If you have questions, feel free to reach out in the GraphRAG discord server: 
https://discord.gg/graphrag